[View in Colaboratory](https://colab.research.google.com/github/sumitdua10/CNN_Text_Classification_Restaurent_Reviews/blob/master/Restnt_Reviews_BoW_Vectorizer.ipynb)

In [3]:
# Natural Language Processing to classify the restaurent reviews whether they are favorable or not. Uses a supervised Learning algorithm
# Main File. It uses a Bags of words approach.

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# 1. Importing the dataset

FILENAME = "https://raw.githubusercontent.com/sumitdua10/CNN_Text_Classification_Restaurent_Reviews/master/Restaurent_reviews.tsv"
#TestFile = FILENAME = "C:\\Users\\IBM_ADMIN\\Desktop\\Personal\\Trainings\\Machine Learning\\Data\\Udemy\\Machine Learning A-Z Template Folder\\Part 7 - Natural Language Processing\\Section 36 - Natural Language Processing\\test.tsv"
dataset = pd.read_csv(FILENAME, delimiter = '\t', error_bad_lines=False)#, quoting = 3, skipfooter=500)#, header = None, skiprows=1)
Num_Words = dataset.shape[0]
print(dataset.head())
print(dataset.shape)


                                              Review  Liked
0                           Wow... Loved this place.      1
1                                 Crust is not good.      0
2          Not tasty and the texture was just nasty.      0
3  Stopped by during the late May bank holiday of...      1
4  The selection on the menu was great and so wer...      1
(1000, 2)


In [4]:


# 2. Download the stop words
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
s = stopwords.words('english')
print(len(s))

#print(type(s))
s.remove('not')
print(len(s))



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
179
178


In [5]:

#corpus.append(review)
#print(corpus)
# 3. Remove th  e puncuation symbols or any other symbols that are not characters [^A-Za-z] and put the text in list Corpus
# 3. Optionaly you can also stem them using porterstemmer. This part of code might be commented below.
print("Original text: ", dataset['Review'][0])
corpus = []
for i in range(0, Num_Words):
    review = re.sub('[^a-zA-Z]', ' ', dataset['Review'][i])
    review = review.lower()
    #review = review.split()
    #ps = PorterStemmer()
    #review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    #review = ' '.join(review)
    corpus.append(review)


#print(len(corpus))
print("First text after removing puncatation: ", corpus[0])
#print(corpus.size)
#print(corpus.shape)
#print(corpus[0])
#corpus.remove((corpus[0]))


Original text:  Wow... Loved this place.
First text after removing puncatation:  wow    loved this place 


In [162]:

# 4. Create the Bag of Words model using CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
cv = CountVectorizer(ngram_range = (1,3)).fit(corpus) # creates features columns with ngrams of size 1, 2 and 3

fnames = cv.get_feature_names()
print("No. of features from simple vectorizer with (1,3) ngrams = ",len(fnames))

tv = TfidfVectorizer(ngram_range = (1,3), max_df=0.15).fit(corpus) # min_df=1, max_df = 0.5

fnames_tv = tv.get_feature_names()
print("No. of features from tfidf vectorizer with (1,3) ngrams= ",len(fnames_tv))

y = dataset.iloc[:, 1].values # y is the output 1 indicates favorable and 0 indicates not good.

No. of features from simple vectorizer with (1,3) ngrams =  16465
No. of features from tfidf vectorizer with (1,3) ngrams=  16460


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

def train_evaluate(v,y):
  
  X_cv = v.transform(corpus).toarray()
  
  X_train, X_test, y_train, y_test = train_test_split(X_cv, y, test_size = 0.20, random_state=123)

  # Fitting Naive Bayes to the Training set
  import sklearn.naive_bayes as nb
  import sklearn.linear_model as lm

  #classifier =  lm.SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=42)
  #classifier = nb.BernoulliNB()
  classifier =LogisticRegression()#random_state = 1)
  #classifier = GaussianNB()
  classifier.fit(X_train, y_train)

  # Predicting the Test set results
  y_pred = classifier.predict(X_test)
  #y_dec = classifier.decision_function(X_test)
  y_prob = classifier.predict_proba(X_test)
  print("Accuracy Score is ", accuracy_score(y_test, y_pred))
  print("F1 Score is ", f1_score(y_test, y_pred))
  print("ROC AUC Score is ", roc_auc_score(y_test, y_pred))
  print("Top 5 features are :")
  #print(list(classifier.get_params()[:5]))
  index = np.argpartition(classifier.coef_,-5, axis=None)[-5:]
    #print(v.get_feature_names()[index])
  print(list(zip(pd.Series(v.get_feature_names())[index], classifier.coef_[0,index])))
  
  return classifier#print(classifier.coef_.shape)
    #print("ROC AUC Score is ", roc_auc_score(y_test, y_prob))

In [165]:
#5. Split the training set and train on Naive_Byes algorithm
# Splitting the dataset into the Training set and Test set

print("Accuracy score from simple vectorizer is :")
classifier_v= train_evaluate(cv,y)

Accuracy score from simple vectorizer is :
Accuracy Score is  0.785
F1 Score is  0.7981220657276996
ROC AUC Score is  0.7837092731829574
Top 5 features are :
[('awesome', 0.9742135751679531), ('delicious', 1.3975597149973336), ('amazing', 1.260670799655687), ('great', 1.9597132885718827), ('good', 1.488608600774885)]


In [164]:
print("Accuracy score from tfidf vectorizer is :")
classifier_t = train_evaluate(tv,y)

Accuracy score from tfidf vectorizer is :
Accuracy Score is  0.785
F1 Score is  0.7860696517412935
ROC AUC Score is  0.7867167919799498
Top 5 features are :
[('nice', 0.9169707670279265), ('amazing', 1.1696054005138004), ('delicious', 1.3535502853078656), ('good', 1.6475629558732934), ('great', 2.300096582491733)]


In [169]:
inputtext = "food was not that good"
review = re.sub('[^a-zA-Z]', ' ', inputtext)
review = review.lower()

print(review)

x = list()
x = [review]
print(x)
#list.append(review)
#corpus.append(review)
#print(len(corpus))
#print(corpus[-1])
#cv = CountVectorizer()#max_features = 1500)
#X_test_run = cv.fit_transform(corpus).toarray()

x_new = cv.transform(x).toarray()
print(x_new.shape)
#print(X_test_run.shape)
y = classifier_v.predict(x_new)
print("Classifier Response is ",y[-1])


food was not that good
['food was not that good']
(1, 16465)
Classifier Response is  0
